# 00 - 确定性推理基础

本notebook展示LLM推理中的确定性基础概念：

1. **随机种子固定** - 确保可重现性
2. **推理入口不变性** - 相同输入产生相同输出
3. **温度为0时的输出概率不变性** - 确定性采样

这些是理解LLM非确定性问题的前提条件。


In [1]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from typing import List, Dict, Any
import sys
import os

# 添加src目录到路径
sys.path.append('..')

# 导入项目模块
from src.device_manager import get_device
from src.font_config import setup_chinese_fonts, force_chinese_fonts

# 设置中文字体
setup_chinese_fonts()
force_chinese_fonts()

# 设置设备
device = get_device('auto')
print(f"使用设备: {device}")
if device.type == 'mps':
    print("使用Apple Silicon MPS加速")


使用中文字体: Arial Unicode MS
使用中文字体: Arial Unicode MS
使用中文字体: Arial Unicode MS
使用中文字体: Arial Unicode MS
使用中文字体: Arial Unicode MS
使用中文字体: Arial Unicode MS
使用中文字体: Arial Unicode MS
使用设备: mps
使用Apple Silicon MPS加速


## 1. 随机种子固定演示

随机种子是确保可重现性的关键。在深度学习中，多个随机源需要同时控制：
- PyTorch的随机数生成器
- Python的random模块
- NumPy的随机数生成器
- CUDA的随机数生成器（如果使用GPU）


In [ ]:
def set_all_seeds(seed: int = 42):
    """设置所有随机种子"""
    # Python随机种子
    random.seed(seed)
    
    # NumPy随机种子
    np.random.seed(seed)
    
    # PyTorch随机种子
    torch.manual_seed(seed)
    
    # 如果使用CUDA
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    # 如果使用MPS
    if device.type == 'mps':
        torch.mps.manual_seed(seed)
    
    # 确保PyTorch的确定性行为
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"所有随机种子已设置为: {seed}")

def reset_all_seeds():
    """重置所有随机种子"""
    # 重置为随机状态
    random.seed()
    np.random.seed()
    torch.manual_seed(torch.initial_seed())
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(torch.cuda.initial_seed())
    
    if device.type == 'mps':
        # MPS没有initial_seed方法，使用随机种子
        torch.mps.manual_seed(random.randint(0, 2**32-1))
    
    print("所有随机种子已重置为随机状态")


In [3]:
# 演示随机种子固定的效果
print("=== 随机种子固定演示 ===")

# 测试1: 不固定种子
print("\n1. 不固定种子（随机状态）:")
reset_all_seeds()
random_values_1 = []
for i in range(5):
    val = torch.randn(1).item()
    random_values_1.append(val)
    print(f"  第{i+1}次: {val:.6f}")

print("\n2. 固定种子（种子=42）:")
set_all_seeds(42)
random_values_2 = []
for i in range(5):
    val = torch.randn(1).item()
    random_values_2.append(val)
    print(f"  第{i+1}次: {val:.6f}")

print("\n3. 再次固定相同种子（种子=42）:")
set_all_seeds(42)
random_values_3 = []
for i in range(5):
    val = torch.randn(1).item()
    random_values_3.append(val)
    print(f"  第{i+1}次: {val:.6f}")

# 验证可重现性
print("\n=== 可重现性验证 ===")
print(f"第2次和第3次结果相同: {random_values_2 == random_values_3}")
print(f"第1次和第2次结果不同: {random_values_1 != random_values_2}")


=== 随机种子固定演示 ===

1. 不固定种子（随机状态）:


AttributeError: module 'torch.mps' has no attribute 'initial_seed'

## 2. 推理入口不变性演示

推理入口不变性是指：对于相同的输入，在固定随机种子的情况下，应该产生完全相同的输出。这是确定性推理的基本要求。


In [ ]:
class SimpleTransformer(nn.Module):
    """简单的Transformer模型用于演示"""
    
    def __init__(self, vocab_size: int = 1000, d_model: int = 512, nhead: int = 8, 
                 num_layers: int = 2, max_len: int = 128):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.Parameter(torch.randn(max_len, d_model))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=2048, 
            dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        # 输入嵌入
        seq_len = x.size(1)
        x = self.embedding(x) * np.sqrt(self.d_model)
        x = x + self.pos_encoding[:seq_len].unsqueeze(0)
        
        # Transformer编码
        x = self.transformer(x)
        
        # 输出投影
        logits = self.output_proj(x)
        
        return logits

# 创建模型
model = SimpleTransformer().to(device)
model.eval()

print(f"模型已创建，参数量: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
def test_inference_invariance(model, input_ids, num_trials: int = 5):
    """测试推理入口不变性"""
    print(f"\n=== 推理入口不变性测试 ===")
    print(f"输入序列: {input_ids.tolist()}")
    print(f"测试次数: {num_trials}")
    
    results = []
    
    for trial in range(num_trials):
        # 固定种子
        set_all_seeds(42)
        
        # 推理
        with torch.no_grad():
            output = model(input_ids)
            # 获取最后一个token的logits
            last_logits = output[0, -1, :].cpu().numpy()
            results.append(last_logits)
            
        print(f"  第{trial+1}次推理完成")
    
    # 验证结果一致性
    reference = results[0]
    all_same = True
    max_diff = 0.0
    
    for i, result in enumerate(results[1:], 1):
        diff = np.max(np.abs(result - reference))
        max_diff = max(max_diff, diff)
        if diff > 1e-6:
            all_same = False
            print(f"  ❌ 第{i+1}次结果与第1次不同，最大差异: {diff:.2e}")
        else:
            print(f"  ✅ 第{i+1}次结果与第1次相同，最大差异: {diff:.2e}")
    
    print(f"\n=== 总结 ===")
    print(f"所有结果相同: {all_same}")
    print(f"最大差异: {max_diff:.2e}")
    
    return all_same, max_diff

# 创建测试输入
test_input = torch.tensor([[1, 5, 10, 15, 20]], device=device)

# 测试推理入口不变性
is_invariant, max_diff = test_inference_invariance(model, test_input, num_trials=5)


## 3. 温度为0时的输出概率不变性

在LLM推理中，温度参数控制输出的随机性：
- 温度 = 0：确定性采样（贪婪解码）
- 温度 > 0：随机采样，温度越高越随机

当温度为0时，模型应该总是选择概率最高的token，从而产生确定性的输出。


In [ ]:
def sample_with_temperature(logits, temperature: float = 1.0, top_k: int = None, top_p: float = None):
    """带温度的采样函数"""
    if temperature == 0:
        # 贪婪解码（确定性）
        return torch.argmax(logits, dim=-1)
    
    # 应用温度
    logits = logits / temperature
    
    # Top-k过滤
    if top_k is not None:
        top_k = min(top_k, logits.size(-1))
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = float('-inf')
    
    # Top-p过滤
    if top_p is not None:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
        
        # 移除累积概率超过top_p的token
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        
        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        logits[indices_to_remove] = float('-inf')
    
    # 采样
    probs = F.softmax(logits, dim=-1)
    return torch.multinomial(probs, num_samples=1)

def test_temperature_invariance(model, input_ids, temperature: float, num_trials: int = 10):
    """测试温度不变性"""
    print(f"\n=== 温度={temperature}时的输出不变性测试 ===")
    
    results = []
    
    for trial in range(num_trials):
        # 固定种子
        set_all_seeds(42)
        
        # 推理
        with torch.no_grad():
            logits = model(input_ids)
            last_logits = logits[0, -1, :]
            
            # 采样
            next_token = sample_with_temperature(last_logits.unsqueeze(0), temperature=temperature)
            results.append(next_token.item())
            
        print(f"  第{trial+1}次采样: token_id = {results[-1]}")
    
    # 分析结果
    unique_tokens = set(results)
    print(f"\n=== 分析结果 ===")
    print(f"唯一token数量: {len(unique_tokens)}")
    print(f"唯一tokens: {sorted(unique_tokens)}")
    
    if temperature == 0:
        is_deterministic = len(unique_tokens) == 1
        print(f"温度=0时是否确定性: {is_deterministic}")
    else:
        print(f"温度={temperature}时的随机性: {len(unique_tokens)}/{num_trials}")
    
    return results


In [ ]:
# 测试不同温度下的行为
print("=== 温度对输出确定性的影响 ===")

# 温度 = 0（确定性）
results_temp0 = test_temperature_invariance(model, test_input, temperature=0.0, num_trials=10)

# 温度 = 0.5（低随机性）
results_temp05 = test_temperature_invariance(model, test_input, temperature=0.5, num_trials=10)

# 温度 = 1.0（标准随机性）
results_temp1 = test_temperature_invariance(model, test_input, temperature=1.0, num_trials=10)

# 温度 = 2.0（高随机性）
results_temp2 = test_temperature_invariance(model, test_input, temperature=2.0, num_trials=10)


## 4. 可视化温度对输出分布的影响
